In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, Input, BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import SGD, Adam
from keras.utils.training_utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from keras import initializers
import prepare_data as pd
import numpy
import math
import scipy.misc as spm

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from PIL import Image

from lr_multiplier import LearningRateMultiplier

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
# see example https://github.com/keras-team/keras/issues/8649
class MyCbk(Callback):

    def __init__(self, model):
         self.model_to_save = model

    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save('checkpoints/64-9-5-5_128-64-SRCNN_model_at_epoch_%d.h5' % epoch)


In [3]:
def psnr(target, ref):
    # assume RGB image
    target_data = numpy.array(target, dtype=float)
    ref_data = numpy.array(ref, dtype=float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(numpy.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

In [4]:
def model(optimizer='SGD', all_layers=True, processing='GPU', lr=0.0001):
    # lrelu = LeakyReLU(alpha=0.1)                                              
#     SRCNN = Sequential()                                                        
#     SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), init='glorot_uniform',  
#                      activation='relu', border_mode='valid', bias=True, input_shape=(32, 32, 1)))
#     SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), init='glorot_uniform',   
#                      activation='relu', border_mode='same', bias=True))         
#     # SRCNN.add(BatchNormalization())                                           
#     SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5), init='glorot_uniform',    
#                      activation='linear', border_mode='valid', bias=True))      
#     adam = Adam(lr=0.0001)                                                     
# #     sgd = SGD(lr=0.0003)                                                        
#     SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
#     return SRCNN 

    multipliers = {'conv2d_1': 1, 'conv2d_2': 1, 'conv2d_3': 0.1}
    
    optimizer = optimizer.upper()
    processing = processing.upper()
    
    print('Using {}.'.format(optimizer))

    if optimizer == 'SGD' and all_layers:
        opt = LearningRateMultiplier(SGD, lr_multipliers=multipliers, lr=lr)
    elif optimizer == 'SGD':
        opt = SGD(lr=0.0001)
    elif all_layers:
        opt = LearningRateMultiplier(Adam, lr_multipliers=multipliers, lr=lr)
    else:
        opt = Adam(lr=lr)
    # lrelu = LeakyReLU(alpha=0.1)
    
    SRCNN = Sequential()
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, input_shape=(64, 64, 1), name='conv2d_1'))

    SRCNN.add(Conv2D(filters=64, kernel_size=(5, 5), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, name='conv2d_2'))
    
    # SRCNN.add(BatchNormalization())
    SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5),
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='linear', padding='same', use_bias=True,  name='conv2d_3'))
    SRCNN_MULTI = multi_gpu_model(SRCNN, gpus=8)
    if processing == 'GPU':    
        SRCNN_MULTI.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    else:
        SRCNN.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN, SRCNN_MULTI

In [5]:
def predict_model():
    # lrelu = LeakyReLU(alpha=0.1)
    sgd = SGD(lr=0.0001)
    
    SRCNN = Sequential()
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, input_shape=(None, None, 1), name='conv2d_1'))
        
    SRCNN.add(Conv2D(filters=64, kernel_size=(5, 5), 
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='relu', padding='same', use_bias=True, name='conv2d_2'))
        
    # SRCNN.add(BatchNormalization())
    SRCNN.add(Conv2D(filters=1, kernel_size=(5, 5),
                     kernel_initializer='glorot_uniform', bias_initializer='random_uniform',
                     activation='linear', padding='same', use_bias=True,  name='conv2d_3'))
    adam = Adam(lr=0.0003)
    sgd_last = SGD(lr=0.00001)

    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

In [6]:
def train(processing='GPU', optimizer='SGD', use_data_augmentation=False, epochs=200, lr=0.0001):
#     srcnn_model = model()
#     print(srcnn_model.summary())                                                
#     data, label = pd.read_training_data("./crop_train.h5")
#     val_data, val_label = pd.read_training_data("./test.h5")                    
                                                                                
#     checkpoint = ModelCheckpoint("SRCNN_check.h5", monitor='val_loss', verbose=1, save_best_only=True,
#                                  save_weights_only=False, mode='min')           
#     callbacks_list = [checkpoint]                                               
                                                                                
#     srcnn_model.fit(data, label, batch_size=1024,
#                     callbacks=callbacks_list, shuffle=True, nb_epoch=200, verbose=2, validation_split=0.2)

    optimizer = optimizer.upper()
    processing = processing.upper()
    
    srcnn_model, srcnn_multi_model = model(optimizer=optimizer, processing=processing, lr=lr)

    callbacks_list = []
    BS = 1024
    
    if processing == 'GPU' or use_data_augmentation:    
        print(srcnn_multi_model.summary())
        # see example https://github.com/keras-team/keras/issues/8649
        cbk = MyCbk(srcnn_model)
        
    else:
        print(srcnn_model.summary())
        checkpoint = ModelCheckpoint("64SRCNN_check.h5", monitor='val_loss', 
                                     verbose=2, 
                                     save_best_only=True,
                                     save_weights_only=False)
        callbacks_list.append(checkpoint)
        
    data, label = pd.read_training_data("./64crop_train.h5")
    val_data, val_label = pd.read_training_data("./64test.h5")

    max_value = data.max()

    data = data/max_value
    label = label/max_value

    val_data = val_data/max_value
    val_label = val_label/max_value
    
    # construct the training image generator for data augmentation
    datagen = ImageDataGenerator(rotation_range=45,
                                 zoom_range=0.15,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 horizontal_flip=True, 
                                 vertical_flip=True)

    # train the network

    if use_data_augmentation and processing == 'GPU':
        history = srcnn_multi_model.fit_generator(datagen.flow(data, label, batch_size=BS),
                                                    validation_data=(val_data, val_label),
                                                    steps_per_epoch=len(data) // BS,
                                                    epochs=epochs, 
                                                    verbose=2, 
                                                    shuffle=True,
                                                    callbacks=[cbk],
                                                    use_multiprocessing=True)
    elif processing == 'GPU':
        srcnn_multi_model.fit(data, label, batch_size=BS,
                              validation_split=0.2,
    #                       validation_data=(val_data, val_label),
                            shuffle=True, 
                            epochs=epochs, 
                            verbose=2,
                            callbacks=[cbk])
    elif use_data_augmentation:
        history = srcnn_model.fit_generator(datagen.flow(data, label, batch_size=BS),
                                          validation_data=(val_data, val_label),
                                          steps_per_epoch=len(data) // BS,
                                          epochs=epochs, 
                                          verbose=2, 
                                          shuffle=True,
                                          callbacks=[cbk])
    else:
        srcnn_model.fit(data, label, batch_size=BS,
                            validation_split=0.2,
    #                       validation_data=(val_data, val_label),
                            shuffle=True, 
                            epochs=epochs, 
                            verbose=2,
                            callbacks=callbacks_list)

    # srcnn_model.load_weights("m_model_adam.h5")

In [7]:
data, label = pd.read_training_data("./64crop_train.h5")
val_data, val_label = pd.read_training_data("./64test.h5")

max_value = data.max()

data = data/max_value
label = label/max_value

val_data = val_data/max_value
val_label = val_label/max_value

In [8]:
def predict():
#     srcnn_model = model()[0]
    srcnn_model = predict_model()
    srcnn_model.load_weights("checkpoints/64-9-5-5_128-64-SRCNN_model_at_epoch_60.h5")
    IMG_NAME = numpy.fromfile('ground_data/pick/recon/slice_256.b', dtype = 'uint8')
    INPUT_NAME = numpy.fromfile("sub_data/pick/recon/slice_256.b", dtype = 'uint8')
    OUTPUT_NAME = "pre2.png"

    dimension_img = int(IMG_NAME.shape[0]**(1/2))
    dimension_input = int(INPUT_NAME.shape[0]**(1/2))
    
    IMG_NAME = IMG_NAME.reshape((dimension_img, dimension_img))
    INPUT_NAME = INPUT_NAME.reshape((dimension_input, dimension_input))
    
    import cv2
    img = INPUT_NAME.copy()
    shape = IMG_NAME.shape
    Y_img = img.copy()
    Y_img = spm.imresize(Y_img, size=shape, interp='bicubic')
    bicubic = Y_img.copy()

    Y_img = Y_img.reshape((dimension_img, dimension_img, 1))
    
    for i in range(Y_img.shape[-1]):
        Y = numpy.zeros((1, shape[0], shape[1], 1), dtype=float)
        Y[0,:,:,0] = Y_img[:,:,i].astype(float) / 255.
        pre = srcnn_model.predict(Y, batch_size=1) * 255.
        
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(float)
    print(Y_img.shape, pre.shape)
    Y_img[:, :, 0] = pre[0, :, :, 0].copy()
    cv2.imwrite(OUTPUT_NAME, Y_img)

    # psnr calculation:
    im1 = IMG_NAME
    im2 = bicubic
    im3 = cv2.imread(OUTPUT_NAME)[:,:,0]
    
    print(im1.shape, im2.shape, im3.shape)

    print ("bicubic:")
    print (cv2.PSNR(im1, im2))
    print ("SRCNN:")
    print (cv2.PSNR(im1, im3))
    
    import matplotlib.pyplot as plt
    print(im1.shape)
    plt.figure(figsize=[15,30])
    plt.subplot(311)
#     plt.imshow(im1[350:450,200:400])
    plt.title('Original')
    plt.imshow(im1)
    plt.subplot(312)
#     plt.imshow(im2[350:450,200:400])
    plt.title('Bicubic')    
    plt.imshow(im2)
    plt.subplot(313)
#     plt.imshow(im3[350:450,200:400])
    plt.title('SRCNN')
    plt.imshow(im3[10:-10,10:-10])
    plt.show()

In [ ]:
train(processing='GPU', optimizer='adam', use_data_augmentation=False, epochs=1000, lr=0.0001)

Using ADAM.
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_1_input (InputLayer)     (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 64, 64, 1)    0           conv2d_1_input[0][0]             
_________________________

Epoch 28/1000
 - 24s - loss: 3.2028e-04 - mean_squared_error: 3.2028e-04 - val_loss: 2.6516e-05 - val_mean_squared_error: 2.6516e-05
Epoch 29/1000
 - 25s - loss: 3.2143e-04 - mean_squared_error: 3.2143e-04 - val_loss: 2.5747e-05 - val_mean_squared_error: 2.5747e-05
Epoch 30/1000
 - 24s - loss: 3.1983e-04 - mean_squared_error: 3.1983e-04 - val_loss: 2.9993e-05 - val_mean_squared_error: 2.9993e-05
Epoch 31/1000
 - 23s - loss: 3.2118e-04 - mean_squared_error: 3.2118e-04 - val_loss: 2.5988e-05 - val_mean_squared_error: 2.5988e-05
Epoch 32/1000
 - 24s - loss: 3.2062e-04 - mean_squared_error: 3.2062e-04 - val_loss: 2.5773e-05 - val_mean_squared_error: 2.5773e-05
Epoch 33/1000
 - 23s - loss: 3.2125e-04 - mean_squared_error: 3.2125e-04 - val_loss: 2.9385e-05 - val_mean_squared_error: 2.9385e-05
Epoch 34/1000
 - 24s - loss: 3.2027e-04 - mean_squared_error: 3.2027e-04 - val_loss: 2.9482e-05 - val_mean_squared_error: 2.9482e-05
Epoch 35/1000
 - 24s - loss: 3.2136e-04 - mean_squared_error: 3.2136e

Epoch 90/1000
 - 23s - loss: 3.1858e-04 - mean_squared_error: 3.1858e-04 - val_loss: 2.6001e-05 - val_mean_squared_error: 2.6001e-05
Epoch 91/1000
 - 24s - loss: 3.1806e-04 - mean_squared_error: 3.1806e-04 - val_loss: 2.5582e-05 - val_mean_squared_error: 2.5582e-05
Epoch 92/1000
 - 25s - loss: 3.1826e-04 - mean_squared_error: 3.1826e-04 - val_loss: 2.5807e-05 - val_mean_squared_error: 2.5807e-05
Epoch 93/1000
 - 24s - loss: 3.1785e-04 - mean_squared_error: 3.1785e-04 - val_loss: 2.5713e-05 - val_mean_squared_error: 2.5713e-05
Epoch 94/1000
 - 25s - loss: 3.1769e-04 - mean_squared_error: 3.1769e-04 - val_loss: 2.7951e-05 - val_mean_squared_error: 2.7951e-05
Epoch 95/1000
 - 24s - loss: 3.1751e-04 - mean_squared_error: 3.1751e-04 - val_loss: 2.8914e-05 - val_mean_squared_error: 2.8914e-05
Epoch 96/1000
 - 25s - loss: 3.1792e-04 - mean_squared_error: 3.1792e-04 - val_loss: 2.5476e-05 - val_mean_squared_error: 2.5476e-05
Epoch 97/1000
 - 25s - loss: 3.1734e-04 - mean_squared_error: 3.1734e

Epoch 152/1000
 - 23s - loss: 3.1308e-04 - mean_squared_error: 3.1308e-04 - val_loss: 2.6208e-05 - val_mean_squared_error: 2.6208e-05
Epoch 153/1000
 - 24s - loss: 3.1304e-04 - mean_squared_error: 3.1304e-04 - val_loss: 2.6543e-05 - val_mean_squared_error: 2.6543e-05
Epoch 154/1000
 - 25s - loss: 3.1316e-04 - mean_squared_error: 3.1316e-04 - val_loss: 2.5126e-05 - val_mean_squared_error: 2.5126e-05
Epoch 155/1000
 - 24s - loss: 3.1312e-04 - mean_squared_error: 3.1312e-04 - val_loss: 2.5172e-05 - val_mean_squared_error: 2.5172e-05
Epoch 156/1000
 - 25s - loss: 3.1306e-04 - mean_squared_error: 3.1306e-04 - val_loss: 2.5327e-05 - val_mean_squared_error: 2.5327e-05
Epoch 157/1000
 - 25s - loss: 3.1292e-04 - mean_squared_error: 3.1292e-04 - val_loss: 2.7011e-05 - val_mean_squared_error: 2.7011e-05
Epoch 158/1000
 - 24s - loss: 3.1288e-04 - mean_squared_error: 3.1288e-04 - val_loss: 2.5499e-05 - val_mean_squared_error: 2.5499e-05
Epoch 159/1000
 - 24s - loss: 3.1297e-04 - mean_squared_error:

Epoch 214/1000
 - 24s - loss: 3.1158e-04 - mean_squared_error: 3.1158e-04 - val_loss: 2.8830e-05 - val_mean_squared_error: 2.8830e-05
Epoch 215/1000
 - 24s - loss: 3.1155e-04 - mean_squared_error: 3.1155e-04 - val_loss: 2.5790e-05 - val_mean_squared_error: 2.5790e-05
Epoch 216/1000
 - 24s - loss: 3.1178e-04 - mean_squared_error: 3.1178e-04 - val_loss: 2.4999e-05 - val_mean_squared_error: 2.4999e-05
Epoch 217/1000
 - 24s - loss: 3.1154e-04 - mean_squared_error: 3.1154e-04 - val_loss: 2.5497e-05 - val_mean_squared_error: 2.5497e-05
Epoch 218/1000
 - 24s - loss: 3.1145e-04 - mean_squared_error: 3.1145e-04 - val_loss: 2.6079e-05 - val_mean_squared_error: 2.6079e-05
Epoch 219/1000
 - 24s - loss: 3.1175e-04 - mean_squared_error: 3.1175e-04 - val_loss: 2.5234e-05 - val_mean_squared_error: 2.5234e-05
Epoch 220/1000
 - 24s - loss: 3.1138e-04 - mean_squared_error: 3.1138e-04 - val_loss: 2.4750e-05 - val_mean_squared_error: 2.4750e-05
Epoch 221/1000
 - 24s - loss: 3.1153e-04 - mean_squared_error:

...

Epoch 462/1000
 - 28s - loss: 3.0943e-04 - mean_squared_error: 3.0943e-04 - val_loss: 2.4850e-05 - val_mean_squared_error: 2.4850e-05
Epoch 463/1000
 - 26s - loss: 3.0945e-04 - mean_squared_error: 3.0945e-04 - val_loss: 2.5660e-05 - val_mean_squared_error: 2.5660e-05
Epoch 464/1000
 - 28s - loss: 3.0933e-04 - mean_squared_error: 3.0933e-04 - val_loss: 2.4735e-05 - val_mean_squared_error: 2.4735e-05
Epoch 465/1000
 - 27s - loss: 3.0940e-04 - mean_squared_error: 3.0940e-04 - val_loss: 2.5347e-05 - val_mean_squared_error: 2.5347e-05
Epoch 466/1000
 - 27s - loss: 3.0950e-04 - mean_squared_error: 3.0950e-04 - val_loss: 2.4791e-05 - val_mean_squared_error: 2.4791e-05
Epoch 467/1000
 - 28s - loss: 3.0940e-04 - mean_squared_error: 3.0940e-04 - val_loss: 2.4606e-05 - val_mean_squared_error: 2.4606e-05
Epoch 468/1000
 - 26s - loss: 3.0952e-04 - mean_squared_error: 3.0952e-04 - val_loss: 2.5793e-05 - val_mean_squared_error: 2.5793e-05
Epoch 469/1000
 - 27s - loss: 3.0963e-04 - mean_squared_error:

Epoch 524/1000
 - 25s - loss: 3.0910e-04 - mean_squared_error: 3.0910e-04 - val_loss: 2.6789e-05 - val_mean_squared_error: 2.6789e-05
Epoch 525/1000
 - 28s - loss: 3.0904e-04 - mean_squared_error: 3.0904e-04 - val_loss: 2.5069e-05 - val_mean_squared_error: 2.5069e-05
Epoch 526/1000
 - 26s - loss: 3.0908e-04 - mean_squared_error: 3.0908e-04 - val_loss: 2.5505e-05 - val_mean_squared_error: 2.5505e-05
Epoch 527/1000
 - 28s - loss: 3.0918e-04 - mean_squared_error: 3.0918e-04 - val_loss: 2.4712e-05 - val_mean_squared_error: 2.4712e-05
Epoch 528/1000
 - 26s - loss: 3.0898e-04 - mean_squared_error: 3.0898e-04 - val_loss: 2.4823e-05 - val_mean_squared_error: 2.4823e-05
Epoch 529/1000
 - 27s - loss: 3.0898e-04 - mean_squared_error: 3.0898e-04 - val_loss: 2.4958e-05 - val_mean_squared_error: 2.4958e-05
Epoch 530/1000
 - 26s - loss: 3.0911e-04 - mean_squared_error: 3.0911e-04 - val_loss: 2.4809e-05 - val_mean_squared_error: 2.4809e-05
Epoch 531/1000
 - 26s - loss: 3.0916e-04 - mean_squared_error:

Epoch 586/1000
 - 28s - loss: 3.0896e-04 - mean_squared_error: 3.0896e-04 - val_loss: 2.5915e-05 - val_mean_squared_error: 2.5915e-05
Epoch 587/1000
 - 26s - loss: 3.0883e-04 - mean_squared_error: 3.0883e-04 - val_loss: 2.4828e-05 - val_mean_squared_error: 2.4828e-05
Epoch 588/1000
 - 27s - loss: 3.0888e-04 - mean_squared_error: 3.0888e-04 - val_loss: 2.6479e-05 - val_mean_squared_error: 2.6479e-05
Epoch 589/1000
 - 26s - loss: 3.0878e-04 - mean_squared_error: 3.0878e-04 - val_loss: 2.5650e-05 - val_mean_squared_error: 2.5650e-05
Epoch 590/1000
 - 28s - loss: 3.0889e-04 - mean_squared_error: 3.0889e-04 - val_loss: 2.4649e-05 - val_mean_squared_error: 2.4649e-05
Epoch 591/1000
 - 29s - loss: 3.0891e-04 - mean_squared_error: 3.0891e-04 - val_loss: 2.4764e-05 - val_mean_squared_error: 2.4764e-05
Epoch 592/1000
 - 28s - loss: 3.0866e-04 - mean_squared_error: 3.0866e-04 - val_loss: 2.4567e-05 - val_mean_squared_error: 2.4567e-05
Epoch 593/1000
 - 26s - loss: 3.0889e-04 - mean_squared_error:

Epoch 648/1000
 - 28s - loss: 3.0870e-04 - mean_squared_error: 3.0870e-04 - val_loss: 2.5210e-05 - val_mean_squared_error: 2.5210e-05
Epoch 649/1000
 - 29s - loss: 3.0860e-04 - mean_squared_error: 3.0860e-04 - val_loss: 2.4802e-05 - val_mean_squared_error: 2.4802e-05
Epoch 650/1000
 - 26s - loss: 3.0868e-04 - mean_squared_error: 3.0868e-04 - val_loss: 2.4933e-05 - val_mean_squared_error: 2.4933e-05
Epoch 651/1000
 - 27s - loss: 3.0869e-04 - mean_squared_error: 3.0869e-04 - val_loss: 2.4936e-05 - val_mean_squared_error: 2.4936e-05
Epoch 652/1000
 - 28s - loss: 3.0844e-04 - mean_squared_error: 3.0844e-04 - val_loss: 2.5465e-05 - val_mean_squared_error: 2.5465e-05
Epoch 653/1000
 - 26s - loss: 3.0869e-04 - mean_squared_error: 3.0869e-04 - val_loss: 2.7520e-05 - val_mean_squared_error: 2.7520e-05
Epoch 654/1000
 - 29s - loss: 3.0878e-04 - mean_squared_error: 3.0878e-04 - val_loss: 2.5600e-05 - val_mean_squared_error: 2.5600e-05
Epoch 655/1000
 - 26s - loss: 3.0854e-04 - mean_squared_error:

Epoch 710/1000
 - 28s - loss: 3.0863e-04 - mean_squared_error: 3.0863e-04 - val_loss: 2.4784e-05 - val_mean_squared_error: 2.4784e-05
Epoch 711/1000
 - 25s - loss: 3.0860e-04 - mean_squared_error: 3.0860e-04 - val_loss: 2.4954e-05 - val_mean_squared_error: 2.4954e-05
Epoch 712/1000
 - 27s - loss: 3.0841e-04 - mean_squared_error: 3.0841e-04 - val_loss: 2.4873e-05 - val_mean_squared_error: 2.4873e-05
Epoch 713/1000
 - 26s - loss: 3.0877e-04 - mean_squared_error: 3.0877e-04 - val_loss: 2.4926e-05 - val_mean_squared_error: 2.4926e-05
Epoch 714/1000
 - 28s - loss: 3.0827e-04 - mean_squared_error: 3.0827e-04 - val_loss: 2.4709e-05 - val_mean_squared_error: 2.4709e-05
Epoch 715/1000
 - 26s - loss: 3.0857e-04 - mean_squared_error: 3.0857e-04 - val_loss: 2.4756e-05 - val_mean_squared_error: 2.4756e-05
Epoch 716/1000
 - 26s - loss: 3.0839e-04 - mean_squared_error: 3.0839e-04 - val_loss: 2.4560e-05 - val_mean_squared_error: 2.4560e-05
Epoch 717/1000
 - 28s - loss: 3.0858e-04 - mean_squared_error:

In [ ]:
predict()

In [ ]:
a = mpimg.imread('butterfly_GT.bmp')
plt.imshow(a)
del(a)